In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables/alltables2016/internationaltradeinservices2016.xls'
    inputFile = sourceFolder / 'internationaltradeinservices2016.xlsb'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table C7 2013-2016')[0]

Loading in\internationaltradeinservices2016.xlsb which has size 750080 bytes
Table names: ['Table C7 2013-2016']


In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo1

{<A73 'Merchanting and Other Trade related Services'>, <A88 'Technical and Scientific Services'>, <A49 'Telecommunication, Computer and Information Services'>, <A36 'Intellectual Property'>, <A5 'Agricultural and Mining Services'>, <A16 'Business and Professional Services'>, <A58 'Construction Services'>, <A68 'Insurance and Pension Services'>, <A10 'Manufacturing, Maintenance and On-site Processing Services'>, <A79 'Personal, Cultural and Recreational  Services'>, <A30 'Research and Development Services'>, <A64 'Financial Services'>}

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo3

{<C23 'Operating leasing services'>, <C11 'Waste treatment and depolution services'>, <C51 'Telecommunications'>, <C82 'Training and educational services'>, <C81 'Medical Services'>, <C91 'Scientific and other techinical services inc surveying'>, <C21 'Recruitment services'>, <C41 'Outright sales and purchases '>, <C27 'Services between related enterprises'>, <C18 'Advertising, market research and public opinion polling services'>, <C52 'Computer Services'>, <C12 'Manufacturing services on goods owned by others'>, <C39 'Charges or payments for the use of '>, <C65 'Financial'>, <C80 'Audio- Visual and related services '>, <C59 'Construction in the UK'>, <C75 'Other trade - related services'>, <C69 'Insurance and Pension Services Claims'>, <C25 'Property management services'>, <C95 'Other trade in services'>, <C31 'Provision of R&D services'>, <C20 'Public relations services'>, <C22 'Legal services'>, <C44 'Outright sales and purchases '>, <C83 'Heritage and recreational services'>, <C90

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec72.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
20,Whole world,2013,balance,Agricultural and Mining Services-Mining and oi...,BOP,GBP Total,5.40711,gbp-million
24,Whole world,2013,exports,Agricultural and Mining Services-Total,BOP,GBP Total,5.41334,gbp-million
32,Whole world,2013,balance,Agricultural and Mining Services-Total,BOP,GBP Total,5.4071,gbp-million
48,Whole world,2013,exports,"Manufacturing, Maintenance and On-site Process...",BOP,GBP Total,5.11664,gbp-million
51,Whole world,2016,exports,"Manufacturing, Maintenance and On-site Process...",BOP,GBP Total,1,gbp-million
